In [2]:
import requests
import os
import tarfile

In [3]:
# https://stackoverflow.com/questions/25010369/wget-curl-large-file-from-google-drive/25033499#25033499
def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://drive.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

In [4]:
destination = '../data.tar.gz'
file_id = '1QkBngulbKYSyYgGeawM9TYfnfAVv4Chl'
if not os.path.isfile(destination):
    download_file_from_google_drive(file_id, destination)

In [5]:
my_tar = tarfile.open('../data.tar.gz')
my_tar.extractall('../')
my_tar.close()

In [6]:
print('# File sizes')
p = '../data'
for f in os.listdir(p):
    if not os.path.isdir(p+'/'+ f):
        print (f.ljust(30) + str(round(os.path.getsize(p+'/'+ f) / 1000000, 2)) + 'MB')
    else:
        sizes = [os.path.getsize(p +'/'+f + '/'+x)/1000000 for x in os.listdir(p+'/'+f)]
        print(f.ljust(30) + str(round(sum(sizes), 2)) + 'MB' + '({} files)'.format(len(sizes)))

# File sizes
models                        1939.14MB(6 files)
densenet169.pkl               57.77MB
resnet101.pkl                 179.1MB
resnet152.pkl                 241.92MB
train_v2.csv                  1.43MB
test_labels.csv               2.35MB
train-jpg                     634.68MB(40479 files)
resnet50.pkl                  102.86MB
test-jpg                      958.88MB(61191 files)
densenet121.pkl               32.64MB
submissions                   16.89MB(7 files)


In [7]:
from __future__ import division
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, merge
from keras.layers import Input, Activation, Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
#from multi_gpu import make_parallel #Available here https://github.com/kuza55/keras-extras/blob/master/utils/multi_gpu.py
from keras.layers.normalization import BatchNormalization
from keras import backend as K
import os
from sklearn.utils import shuffle
import random
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import fbeta_score
from keras.optimizers import Adam, SGD

os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

In [8]:
def fbeta_loss(y_true, y_pred):
    beta_squared = 4

    tp = K.sum(y_true * y_pred) + K.epsilon()
    fp = K.sum(y_pred) - tp
    fn = K.sum(y_true) - tp

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    result = 1 - (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

    return result

In [9]:
def fbeta_score_K(y_true, y_pred):
    beta_squared = 4

    tp = K.sum(y_true * y_pred) + K.epsilon()
    fp = K.sum(y_pred) - tp
    fn = K.sum(y_true) - tp

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    result = (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

    return result

In [10]:
def rotate(img):
    rows = img.shape[0]
    cols = img.shape[1]
    angle = np.random.choice((10, 20, 30))#, 40, 50, 60, 70, 80, 90))
    rotation_M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    img = cv2.warpAffine(img, rotation_M, (cols, rows))
    return img

In [11]:
def rotate_bound(image, size):
    #credits http://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    angle = np.random.randint(10,180)

    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    output = cv2.resize(cv2.warpAffine(image, M, (nW, nH)), (size, size))
    return output

In [12]:
def perspective(img):
    rows = img.shape[0]
    cols = img.shape[1]

    shrink_ratio1 = np.random.randint(low=85, high=110, dtype=int) / 100
    shrink_ratio2 = np.random.randint(low=85, high=110, dtype=int) / 100

    zero_point = rows - np.round(rows * shrink_ratio1, 0)
    max_point_row = np.round(rows * shrink_ratio1, 0)
    max_point_col = np.round(cols * shrink_ratio2, 0)

    src = np.float32([[zero_point, zero_point], [max_point_row-1, zero_point], [zero_point, max_point_col+1], [max_point_row-1, max_point_col+1]])
    dst = np.float32([[0, 0], [rows, 0], [0, cols], [rows, cols]])

    perspective_M = cv2.getPerspectiveTransform(src, dst)

    img = cv2.warpPerspective(img, perspective_M, (cols,rows))#, borderValue=mean_pix)
    return img

In [13]:
def shift(img):
    rows = img.shape[0]
    cols = img.shape[1]

    shift_ratio1 = (random.random() * 2 - 1) * np.random.randint(low=3, high=15, dtype=int)
    shift_ratio2 = (random.random() * 2 - 1) * np.random.randint(low=3, high=15, dtype=int)

    shift_M = np.float32([[1,0,shift_ratio1], [0,1,shift_ratio2]])
    img = cv2.warpAffine(img, shift_M, (cols, rows))#, borderValue=mean_pix)
    return img

In [14]:
def batch_generator_train(zip_list, img_size, batch_size, is_train=True, shuffle=True):
    number_of_batches = np.ceil(len(zip_list) / batch_size)
    if shuffle == True:
        random.shuffle(zip_list)
    counter = 0
    while True:
        if shuffle == True:
            random.shuffle(zip_list)

        batch_files = zip_list[batch_size*counter:batch_size*(counter+1)]
        image_list = []
        mask_list = []

        for file, mask in batch_files:

            image = cv2.resize(cv2.imread(file), (img_size,img_size)) / 255.
            image = image[:, :, [2, 1, 0]] - mean_pix

            rnd_flip = np.random.randint(2, dtype=int)
            rnd_rotate = np.random.randint(2, dtype=int)
            rnd_zoom = np.random.randint(2, dtype=int)
            rnd_shift = np.random.randint(2, dtype=int)

            if (rnd_flip == 1) & (is_train == True):
                rnd_flip = np.random.randint(3, dtype=int) - 1
                image = cv2.flip(image, rnd_flip)

            if (rnd_rotate == 1) & (is_train == True):
                image = rotate_bound(image, img_size)

            if (rnd_zoom == 1) & (is_train == True):
                image = perspective(image)

            if (rnd_shift == 1) & (is_train == True):
                image = shift(image)

            image_list.append(image)
            mask_list.append(mask)

        counter += 1
        image_list = np.array(image_list)
        mask_list = np.array(mask_list)

        yield (image_list, mask_list)

        if counter == number_of_batches:
            if shuffle == True:
                random.shuffle(zip_list)
            counter = 0

def batch_generator_test(zip_list, img_size, batch_size, shuffle=True):
    number_of_batches = np.ceil(len(zip_list)/batch_size)
    print(len(zip_list), number_of_batches)
    counter = 0
    if shuffle:
        random.shuffle(zip_list)
    while True:
        batch_files = zip_list[batch_size*counter:batch_size*(counter+1)]
        image_list = []
        mask_list = []

        for file, mask in batch_files:

            image = cv2.resize(cv2.imread(file), (img_size, img_size))
            image = image[:, :, [2, 1, 0]] - mean_pix
            image_list.append(image)
            mask_list.append(mask)

        counter += 1
        image_list = np.array(image_list)
        mask_list = np.array(mask_list)

        yield (image_list, mask_list)

        if counter == number_of_batches:
            random.shuffle(zip_list)
            counter = 0


In [16]:
def predict_generator(files, img_size, batch_size):
    number_of_batches = np.ceil(len(files) / batch_size)
    print(len(files), number_of_batches)
    counter = 0
    int_counter = 0

    while True:
            beg = batch_size * counter
            end = batch_size * (counter + 1)
            batch_files = files[beg:end]
            image_list = []

            for file in batch_files:
                int_counter += 1
                image = cv2.resize(cv2.imread(file), (img_size, img_size))
                image = image[:, :, [2, 1, 0]] - mean_pix

                rnd_flip = np.random.randint(2, dtype=int)
                rnd_rotate = np.random.randint(2, dtype=int)
                rnd_zoom = np.random.randint(2, dtype=int)
                rnd_shift = np.random.randint(2, dtype=int)

                if rnd_flip == 1:
                    rnd_flip = np.random.randint(3, dtype=int) - 1
                    image = cv2.flip(image, rnd_flip)

                if rnd_rotate == 1:
                    image = rotate_bound(image, img_size)

                if rnd_zoom == 1:
                    image = perspective(image)

                if rnd_shift == 1:
                    image = shift(image)

                image_list.append(image)

            counter += 1

            image_list = np.array(image_list)

            yield (image_list)


def f2_score(y_true, y_pred):
    y_true, y_pred, = np.array(y_true), np.array(y_pred)
    score = fbeta_score(y_true, y_pred, beta=2, average='samples')
    return score

In [17]:
GLOBAL_PATH = '../data/'
TRAIN_FOLDER = '../data/train-jpg/'
TEST_FOLDER = '../data/test-jpg/' 
F_CLASSES = GLOBAL_PATH + 'train_v2.csv'

In [18]:
df_train = pd.read_csv(F_CLASSES)
df_test = pd.read_csv(GLOBAL_PATH + 'test_labels.csv')

In [19]:
labels = ['blow_down',
          'bare_ground',
          'conventional_mine',
          'blooming',
          'cultivation',
          'artisinal_mine',
          'haze',
          'primary',
          'slash_burn',
          'habitation',
          'clear',
          'road',
          'selective_logging',
          'partly_cloudy',
          'agriculture',
          'water',
          'cloudy']

In [20]:
label_map = {'agriculture': 14,
             'artisinal_mine': 5,
             'bare_ground': 1,
             'blooming': 3,
             'blow_down': 0,
             'clear': 10,
             'cloudy': 16,
             'conventional_mine': 2,
             'cultivation': 4,
             'habitation': 9,
             'haze': 6,
             'partly_cloudy': 13,
             'primary': 7,
             'road': 11,
             'selective_logging': 12,
             'slash_burn': 8,
             'water': 15}

In [21]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [22]:
x_train = []
x_test = []
y_train = []

In [23]:
for f, tags in tqdm(df_train.values, miniters=1000):
    img = TRAIN_FOLDER + '{}.jpg'.format(f)
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    x_train.append(img)
    y_train.append(targets)

100%|██████████| 40479/40479 [00:00<00:00, 281279.64it/s]


In [24]:
x_train, x_holdout, y_train, y_holdout = x_train[3000:-1], x_train[:3000], y_train[3000:-1], y_train[:3000]

x_train, y_train = shuffle(x_train, y_train, random_state = 24)

In [25]:
part = 0.85
split = int(round(part*len(y_train)))
x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]
print('x tr: ', len(x_train))

x tr:  31856


In [26]:
#define callbacks
callbacks = [ModelCheckpoint('amazon_2007.hdf5', monitor='val_loss', save_best_only=True, verbose=2, save_weights_only=False),
             ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0.0000001),
             EarlyStopping(monitor='val_loss', patience=5, verbose=0)]

In [27]:
BATCH = 128
IMG_SIZE = 224
mean_pix = np.array([102.9801, 115.9465, 122.7717]) #It is BGR

In [28]:
from keras.applications import ResNet50

In [29]:
#Compile model and set non-top layets non-trainable (warm-up)
base_model = ResNet50(include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3), pooling='avg', weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
output = Dense(17, activation='sigmoid')(x)

optimizer = Adam(0.001, decay=0.0003)
model = Model(inputs=base_model.inputs, outputs=output)


model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', fbeta_score_K])


94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
model.fit_generator(generator=batch_generator_train(list(zip(x_train, y_train)), IMG_SIZE, BATCH),
                          steps_per_epoch=np.ceil(len(x_train)/BATCH),
                          epochs=1,
                          verbose=1,
                          validation_data=batch_generator_train(list(zip(x_valid, y_valid)), IMG_SIZE, 16),
                          validation_steps=np.ceil(len(x_valid)/16),
                          callbacks=callbacks,
                          initial_epoch=0)   

In [34]:
#Compile model and set all layers trainable
from keras import optimizers
optimizer = optimizers.Adam(0.0001, decay=0.00000001)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', fbeta_score_K])
model.load_weights('amazon_2007.hdf5', by_name=True)
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
BATCH = 32
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', fbeta_score_K])
model.fit_generator(generator=batch_generator_train(list(zip(x_train, y_train)), IMG_SIZE, BATCH),
                          steps_per_epoch=np.ceil(len(x_train)/BATCH),
                          epochs=50,
                          verbose=1,
                          validation_data=batch_generator_train(list(zip(x_valid, y_valid)), IMG_SIZE, 16),
                          validation_steps=np.ceil(len(x_valid)/16),
                          callbacks=callbacks,
                          initial_epoch=0)

In [36]:
model.load_weights('amazon_2007.hdf5')

In [37]:
x_val = []
y_val = []
x_hld = []
y_hld = []
x_test = []
y_test = []

for f, tags in tqdm(list(zip(x_valid, y_valid)), miniters=1000):
    y_val.append(tags) 

100%|██████████| 5622/5622 [00:00<00:00, 1178370.75it/s]


In [38]:
p_valid = model.predict_generator(batch_generator_test(list(zip(x_valid, y_valid)), IMG_SIZE, 8, shuffle=False), steps=np.ceil(len(x_valid)/8))

Instructions for updating:
Please use Model.predict, which supports generators.
5622 703.0


In [39]:
print('val_set: ', fbeta_score(np.array(y_val), np.array(p_valid) > 0.2, beta=2, average='samples'))

val_set:  0.4694372408764788


In [40]:
def optimise_f2_thresholds(y, p, verbose=True, resolution=100):
    #credits https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/discussion/32475
  def mf(x):
    p2 = np.zeros_like(p)
    for i in range(17):
      p2[:, i] = (p[:, i] > x[i]).astype(np.int)
    score = fbeta_score(y, p2, beta=2, average='samples')
    return score

  x = [0.2]*17
  for i in range(17):
    best_i2 = 0
    best_score = 0
    for i2 in range(resolution):
      i2 /= resolution
      x[i] = i2
      score = mf(x)
      if score > best_score:
        best_i2 = i2
        best_score = score
    x[i] = best_i2
    if verbose:
      print(i, best_i2, best_score)

  return x

In [41]:
X = optimise_f2_thresholds(np.array(y_val), np.array(p_valid))

0 0.01 0.4694372408764788
1 0.07 0.4694372408764788
2 0.06 0.4694372408764788
3 0.01 0.4694372408764788
4 0.16 0.4694372408764788
5 0.99 0.4699491249625953
6 0.01 0.4699491249625953
7 0.0 0.4699491249625953
8 0.01 0.4699491249625953
9 0.22 0.46996052705510727
10 0.0 0.6637152918176321
11 0.73 0.6637759302187187
12 0.01 0.6637759302187187
13 0.99 0.6725717809587499
14 0.0 0.7004998983194927
15 0.99 0.7092211437605831
16 0.01 0.7092211437605831


In [42]:
for f, tags in tqdm(list(zip(x_holdout, y_holdout)), miniters=1000):
    img = cv2.resize(cv2.imread(f), (IMG_SIZE, IMG_SIZE))
    x_hld.append(img)
    y_hld.append(tags)

100%|██████████| 3000/3000 [00:06<00:00, 441.56it/s]


In [43]:
if len(x_holdout) % 2 > 0:
    x_hld.append(x_hld[0])
    y_hld.append(y_hld[0])

x_hld = np.array(x_hld, np.float16)

p_valid = model.predict(x_hld, batch_size=28, verbose=2)
print('holdout set: ', f2_score(np.array(y_hld), np.array(p_valid) > 0.2))
print('holdout set w/ thresh: ', f2_score(np.array(y_hld), np.array(p_valid) > 0.19))

108/108 - 9s
holdout set:  0.47312073788907216
holdout set w/ thresh:  0.4732788762906316


In [44]:
for f, tags in tqdm(df_test.values, miniters=1000):
    img = TEST_FOLDER + '{}.jpg'.format(f)
    x_test.append(img)

batch_size_test = 32
len_test = len(x_test)
x_tst = []
yfull_test = []

100%|██████████| 61191/61191 [00:00<00:00, 620752.04it/s]


In [ ]:
TTA_steps = 30

for k in range(0, TTA_steps):
    print(k)
    probs = model.predict_generator(predict_generator(x_test,IMG_SIZE,batch_size_test), steps=np.ceil(len(x_test)/batch_size_test),verbose=1)
    yfull_test.append(probs)
    k += 1

result = np.array(yfull_test[0])

In [ ]:
for i in range(1, TTA_steps):
    result += np.array(yfull_test[i])
result /= TTA_steps

In [123]:
res = pd.DataFrame(result, columns=labels)
preds = []

In [124]:
for i in tqdm(range(res.shape[0]), miniters=1000):
    a = res.loc[[i]] 
    a = a.apply(lambda x: x > X, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

100%|██████████| 61191/61191 [04:02<00:00, 252.61it/s]


In [125]:
print(len(preds))

61191


In [130]:
df_test['tags'] = preds
df_test.to_csv('submission1.csv', index=False)